https://betfair-datascientists.github.io/api/apiPythontutorial/#get-market-books

In [2]:
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json

# Change this certs path to wherever you're storing your certificates

# Your credentials.json file should look like this:

# {
#     "username" : "johnsmith123",
#     "password" : "guest",
#     "app_key" : "****************"
# }

with open('credentials.json') as f:
    cred = json.load(f)
    my_username = cred['username']
    my_password = cred['password']
    my_app_key = cred['app_key']
print(my_username)
certs_path = "/certs"

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key,
                                       cert_files=certs_path)

trading.login()

log_runner


APIError: None 
Params: None 
Exception: ('Connection aborted.', PermissionError(13, 'Permission denied'))

In [9]:
import betfairlightweight
from betfairlightweight import filters


# create trading instance
trading = betfairlightweight.APIClient(my_username, my_password, my_app_key)

# login
trading.login()

# make event type request to find horse racing event type
horse_racing_event_type_id = trading.betting.list_event_types(
    filter=filters.market_filter(text_query="Horse Racing")
)

# returns one result
print(horse_racing_event_type_id)

for event_type in horse_racing_event_type_id:
    # prints id, name and market count
    print(event_type.event_type.id, event_type.event_type.name, event_type.market_count)
    horse_racing_id = event_type.event_type.id

    # list all horse racing market catalogues
    market_catalogues = trading.betting.list_market_catalogue(
        filter=filters.market_filter(
            event_type_ids=[horse_racing_id],  # filter on just horse racing
            market_countries=["GB"],  # filter on just GB countries
            market_type_codes=["WIN"],  # filter on just WIN market types
        ),
        market_projection=[
            "MARKET_START_TIME",
            "RUNNER_DESCRIPTION",
        ],  # runner description required
        max_results=2,
    )

    print("%s market catalogues returned" % len(market_catalogues))

    for market_catalogue in market_catalogues:
        # prints market id, market name and market start time
        print(
            market_catalogue.market_id,
            market_catalogue.market_name,
            market_catalogue.market_start_time,
        )

        for runner in market_catalogue.runners:
            # prints runner id, runner name and handicap
            print(runner.selection_id, runner.runner_name, runner.handicap)

        # market book request
        market_books = trading.betting.list_market_book(
            market_ids=[market_catalogue.market_id],
            price_projection=filters.price_projection(
                price_data=filters.price_data(ex_all_offers=True)
            ),
        )

        for market_book in market_books:
            # prints market id, inplay?, status and total matched
            print(
                market_book.market_id,
                market_book.inplay,
                market_book.status,
                market_book.total_matched,
            )

            for runner in market_book.runners:
                # prints selection id, status and total matched
                print(runner.selection_id, runner.status, runner.total_matched)

                available_to_back = runner.ex.available_to_back
                available_to_lay = runner.ex.available_to_lay
                print(runner.runner_name, available_to_back, available_to_lay)


# logout
trading.logout()

[<EventTypeResult>]
7 Horse Racing 927
2 market catalogues returned
1.215231994 5f Nov Stks 2023-06-16 12:30:00
56496587 Kingswood Flyer 0.0
57120916 Markakol 0.0
55467852 Chinese Knot 0.0
56693215 Nazalan 0.0
33407 Enchanting 0.0
56366476 Notta Nother 0.0
8823386 Blow Me Away 0.0
56330237 Havana Mojito 0.0
1.215231994 False OPEN 4654.14
56496587 ACTIVE 0.0


AttributeError: 'RunnerBook' object has no attribute 'runner_name'

In [2]:
from betfairlightweight.filters import (
    streaming_market_filter,
    streaming_market_data_filter,
)

betfair_socket = trading.streaming.create_stream()

market_filter = streaming_market_filter(
    event_type_ids=['7'],
    country_codes=['IE'],
    market_types=['WIN'],
)
market_data_filter = streaming_market_data_filter(
    fields=['EX_ALL_OFFERS', 'EX_MARKET_DEF'],
    ladder_levels=3
)

betfair_socket.subscribe_to_markets(
    market_filter=market_filter,
    market_data_filter=market_data_filter,
)

betfair_socket.start()  # blocking

KeyboardInterrupt: 

In [26]:
from datetime import datetime, timedelta

# Set the time range for the next 24 hours
start_time = datetime.utcnow()
end_time = start_time + timedelta(hours=24)

# Define the filter for horse racing markets
event_type_ids = ["7"]  # Horse Racing event type ID
market_filter = {
    "eventTypeIds": event_type_ids,
    "marketStartTime": {"from": start_time.isoformat(), "to": end_time.isoformat()},
    "marketTypeCodes": ["WIN"],  # Only retrieve win markets
}

# Get the list of market catalogues
market_catalogues = trading.betting.list_market_catalogue(
    filter=market_filter,
    market_projection=["MARKET_START_TIME", "RUNNER_METADATA"],
    max_results=100,  # Maximum number of market catalogues to retrieve
)

for market_catalogue in market_catalogues:
    market_id = market_catalogue.market_id

    # Get the market book for the market
    market_book = trading.betting.get_market_book(
        market_ids=[market_id],
        price_projection={"priceData": ["EX_BEST_OFFERS"]},
    )

    if market_book:
        # Process the market book
        # ... Your code here ...
        print(market_book)
    else:
        print("Market book not found for market ID:", market_id)

# Logout from the API
trading.logout()


AttributeError: 'Betting' object has no attribute 'get_market_book'